In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import comb, gamma
%matplotlib inline

from ipywidgets import interact, interactive_output, fixed, interact_manual, HBox, Label, VBox, Accordion
import ipywidgets as widgets
from IPython.display import Image
from IPython import display

layout = widgets.Layout(width='auto', height='40px')
style = {'description_width': 'initial'}

In [2]:
def binomial(N,m,q):
    return comb(N,m)*(q**m)*((1-q)**(N-m))

def beta(a,b,q):
    return (gamma(a+b)/(gamma(a)*(gamma(b))))*(q**(a-1))*((1-q)**(b-1))

def posteriori(N,N1,a,b,q):
    return beta(N1+a, N-N1+b, q)

def predict(M,x,a,b):
    return comb(M,x) * (gamma(a+b)/(gamma(a)*gamma(b))) * ((gamma(x+a)*gamma(M-x+b))/gamma(a+b+M))

In [3]:
def plot(N,N1,a,b,M):
    theta = np.linspace(0,1,100)

    priori = beta(a,b,theta)

    vero = binomial(N,N1,theta)

    post = posteriori(N,N1,a,b,theta) 
    
    theta_ml = np.argmax(vero)
    theta_map = np.argmax(post)

    fig = plt.figure(figsize=(15,5))
    fig.suptitle(f"Analisando experimento com {N} observações, sendo {N1} caras", fontsize=15)
#     fig.subplots_adjust(top=0.9)

    
    plt.subplot(1,2,1)
    plt.plot(theta, vero, 'r', label="Verossimilhança")
    plt.plot(theta, priori, 'b', label="Priori")
    plt.plot(theta, post, 'g', label="Posteriori")
    plt.scatter([theta[theta_ml]], [post[theta_ml]], label="theta ML", marker="x", color="r", s=50)
    plt.scatter([theta[theta_map]], [post[theta_map]], label="theta MAP", marker="^", color="g", s=50)
    plt.title("Análise das distribuições a priori e posteriori")
    plt.legend()
    
    plt.subplot(1,2,2)
    
    a_hat = N1 + a
    b_hat = N - N1+ b
    
    x = range(0,M+1,1)
    prediction = np.array([predict(M,x,a_hat,b_hat) for x in x])
    
    plt.bar(x, prediction)
    plt.title(f"Distribuição preditiva de caras após {M} observações")

In [4]:
N = widgets.IntSlider(value=4, min=1, max=10)
N1 = widgets.IntSlider(value=2, min=1, max=10)
M = widgets.IntSlider(value=5, min=1, max=10)
a = widgets.IntSlider(value=2, min=1, max=10, description='a')
b = widgets.IntSlider(value=2, min=1, max=10, description='b')
interactive_plot = interactive_output(plot, 
                               {"N":N,
#                                N=widgets.IntSlider(value=4, min=1, max=10, description="Observações iniciais", style=style), 
#                                N1=widgets.IntSlider(value=2, min=1, max=10, description="Observações iniciais com cara", style=style), 
                               "N1":N1,
                               "a":a, "b":b, 
#                                M=widgets.IntSlider(value=5, min=1, max=10, description="Próximas observações")
                               "M":M}
                              )

acc = Accordion([
    HBox([Label("Total"), N, Label("Caras"), N1]),
    HBox([a,b]),
])

acc.set_title(0,"Observações iniciais")
acc.set_title(1,"Hiperparâmetros")

ui = VBox([
    acc,
    HBox([Label("Observações posteriores"), M])
])

display.display(ui, interactive_plot)

Output()